## Projeto de Machine Learning I

### Luan Fábio Marinho Galindo

#### Introdução ao projeto

Esse projeto utiliza um dataset de frutas  disponível no Kaggle (https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class/data)

O objetivo deste trabalho é resolver um problema de classificação de frutas utilizando algoritmos KNN e Random Forest, além do pacote scikit-learn, 
conforme as instruções do trabalho. 

In [2]:
# Importando as bibliotecas necessárias
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import os

2024-04-29 22:48:34.855524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Primeiramente vamos carregar as imagens de cada tipo (categoria) de fruta das subpastas correspondentes e redimensioná-las para 150x150 pixels. Além disso, os labels serão convertidos para valores numéricos para que possam ser usados em modelos de machine learning. Por fim, será feita a normalização os dados dividindo-os por 255 (tamanho do intervalo RGB), pois estamos trabalhando com imagens.


In [3]:

# Definindo o caminho para a pasta 'archive', que deve se encontrar na mesma pasta do script
base_path = (os.getcwd() + '//archive')

# Categorias das frutas
categories = ['apple', 'avocado', 'banana', 'cherry', 'kiwi', 'mango', 'orange', 'pineapple', 'strawberries', 'watermelon']

# Função para carregar as imagens e os labels
def load_dataset(dataset_type):
    images = []
    labels = []
    # Verificando se é o dataset de predição
    if dataset_type == 'predict':
        path = os.path.join(base_path, dataset_type)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = load_img(img_path, target_size=(150, 150))  # Redimensionando
            img = img_to_array(img)
            images.append(img)
        return np.array(images)
    else:
        for category in categories:
            path = os.path.join(base_path, dataset_type, category)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = load_img(img_path, target_size=(150, 150))
                img = img_to_array(img)
                images.append(img)
                labels.append(category)
        return np.array(images), np.array(labels)

# Carregando os datasets
X_train, y_train = load_dataset('train')
X_test, y_test = load_dataset('test')
X_predict = load_dataset('predict')  # Apenas imagens, sem labels

# Convertendo os labels para valores numéricos
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Dividindo o dataset de treino em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalizando os dados
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [4]:
def load_dataset(dataset_type):
    images = []
    labels = []
    # Verificando se é o dataset de predição
    if dataset_type == 'predict':
        path = os.path.join(base_path, dataset_type)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = load_img(img_path, target_size=(150, 150))  # Redimensionando
            img = img_to_array(img)
            images.append(img)
        return np.array(images)
    else:
        for category in categories:
            path = os.path.join(base_path, dataset_type, category)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = load_img(img_path, target_size=(150, 150))
                img = img_to_array(img)
                images.append(img)
                labels.append(category)
        return np.array(images), np.array(labels)

# Carregando os datasets
X_train, y_train = load_dataset('train')
X_test, y_test = load_dataset('test')
X_predict = load_dataset('predict')  # Apenas imagens, sem labels

# Convertendo os labels para valores numéricos
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Dividindo o dataset de treino em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalizando os dados
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


Os modelos KNeighborsClassifier e RandomForestClassifier esperam receber arrays de no máximo 2 dimensões, o que não acontece com imagens, dado que esses dados possuem 4 dimensões (número de amostras, altura, largura e canal de cor). Para resolver esse problema, devemos fazer uma mudança nas dimensões dos dados para que eles possuam as dimensões corretas para alimentar a KNN e a RandomForest. Podemos realizar um "achatamento"  das imagens, convertendo-as em um vetor unidimensional.

Após isso, faremos os treinamentos dos algoritmos utilizando o Grid Search para otimizar os hiperparâmetros.

In [9]:
# Achatar as imagens para o KNN e RandomForest
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Escalando os dados achatados
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

# KNN
knn = KNeighborsClassifier()
param_grid_knn = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5)
grid_search_knn.fit(X_train_flat, y_train)
knn_best = grid_search_knn.best_estimator_
knn_predictions = knn_best.predict(X_test_flat)

print("Melhores parâmetros para KNN:", grid_search_knn.best_params_)
print(classification_report(y_test, knn_predictions))

Melhores parâmetros para KNN: {'n_neighbors': 9, 'weights': 'distance'}
              precision    recall  f1-score   support

           0       0.46      0.62      0.53        89
           1       0.80      0.04      0.07       105
           2       0.24      0.30      0.27       106
           3       0.61      0.22      0.32       105
           4       0.22      0.55      0.32       105
           5       0.25      0.21      0.23       105
           6       0.39      0.52      0.44        97
           7       0.41      0.73      0.53       105
           8       0.64      0.16      0.25       103
           9       0.33      0.13      0.19       105

    accuracy                           0.34      1025
   macro avg       0.44      0.35      0.31      1025
weighted avg       0.44      0.34      0.31      1025



Notou-se um desempenho bastante fraco, com o melhor parâmetro sendo 9 vizinhos e usando como pesos os vizinhos mais próximos da amostra. Acredito que aumentar o número de vizinhos possa ajudar a melhorar os resultados, mas não farei o teste devido ao provável custo computacional associado.

In [10]:
# RandomForest
rf = RandomForestClassifier()
param_grid_rf = {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, 30]}
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5)
grid_search_rf.fit(X_train_flat, y_train)
rf_best = grid_search_rf.best_estimator_
rf_predictions = rf_best.predict(X_test_flat)

print("Melhores parâmetros para RandomForest:", grid_search_rf.best_params_)
print(classification_report(y_test, rf_predictions))

Melhores parâmetros para RandomForest: {'max_depth': 20, 'n_estimators': 300}
              precision    recall  f1-score   support

           0       0.55      0.61      0.57        89
           1       0.44      0.41      0.42       105
           2       0.39      0.32      0.35       106
           3       0.54      0.58      0.56       105
           4       0.50      0.68      0.57       105
           5       0.35      0.31      0.33       105
           6       0.58      0.59      0.58        97
           7       0.71      0.75      0.73       105
           8       0.51      0.52      0.52       103
           9       0.41      0.29      0.34       105

    accuracy                           0.50      1025
   macro avg       0.50      0.51      0.50      1025
weighted avg       0.49      0.50      0.50      1025



A Floresta Aleatória mostrou melhor desempenho, apesar de ter consumido 23m21.1s para realizar o treinamento. Com máxima profundidade de árvores em 20 (resultando em até 20 níveis em cada árvore) e até 300 árvores na floresta como melhores parâmetros neste teste, podemos concluir que um aumento de árvores independentes da floresta pode ajudar a melhorar os resultados, mas com elevado custo computacional esperado.

Como terceira alternativa, iremos usar uma rede neural convolucional (CNN) simples, pois é um modelo comum para tarefas de visão computacional, para analisar se nossos resultados podem ser melhorados.

In [8]:
# CNN
# Função para criar o modelo Keras
def create_model(optimizer, activation):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation=activation, input_shape=(150, 150, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation=activation))
    model.add(Dense(len(categories), activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Envolver o modelo com KerasClassifier
model_cnn = KerasClassifier(build_fn=create_model, verbose=0)

# Definir os hiperparâmetros para o Grid Search
param_grid = {
    'optimizer': ['rmsprop', 'adam'],
    'activation': ['relu', 'tanh'],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30]
}

# Criar o Grid Search
grid = GridSearchCV(estimator=model_cnn, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X_train, y_train)

# Fazendo predições com o conjunto 'predict'
cnn_predictions = grid_result.best_estimator_.predict(X_predict)
cnn_predicted_classes = np.argmax(cnn_predictions, axis=1)

# Convertendo os valores numéricos de volta para labels categóricos
cnn_predicted_labels = label_encoder.inverse_transform(cnn_predicted_classes)
print("\n\nPredições: ", cnn_predicted_labels)
print("Melhores parâmetros: %s" % grid_result.best_params_)
print("Melhor acurácia: %f" % grid_result.best_score_)

/tmp/ipykernel_5013/1662838897.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_cnn = KerasClassifier(build_fn=create_model, verbose=0)
2024-04-29 23:03:58.872945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 23:03:58.882231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-

KeyboardInterrupt: 

O tempo de computação ficou muito grande nesse algoritmo, por esse motivo não foi possível colher resultados e chegar a conclusões.

In [ ]:
print("Comparação de desempenho entre CNN, KNN e RandomForest:")
print("CNN - Relatório de Classificação:")
print(classification_report(y_test, cnn_predictions))
print("KNN - Relatório de Classificação:")
print(classification_report(y_test, knn_predictions))
print("\n\nRandomForest - Relatório de Classificação:")
print(classification_report(y_test, rf_predictions))

O trabalho chega ao fim com a conclusão parcial de que a Floresta Aleatória mostrou o melhor desempenho, desconsiderando os resultados da CNN.